# NLP HW2 (2021)

## Дарья Родионова БКЛ182

### Collocation extraction + NER

In [1]:
import json
import gzip
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

import nltk
from matplotlib import pyplot as plt
from collections import Counter

import pickle
import string
from nltk.corpus import stopwords
stops = stopwords.words('english')

import spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.util import ngrams
from nltk.collocations import *

lemmatizer = WordNetLemmatizer()

from spacy.matcher import Matcher

### Скачивание и подготовка корпуса

In [2]:
data = []
with open('Musical_Instruments_5.json', 'r') as f:
    for l in tqdm(f):
        data.append(json.loads(l))

In [3]:
len(data)

231392

In [4]:
data[0]

{'overall': 5.0,
 'verified': True,
 'reviewTime': '10 30, 2016',
 'reviewerID': 'A3FO5AKVTFRCRJ',
 'asin': '0739079891',
 'reviewerName': 'francisco',
 'reviewText': "It's good for beginners",
 'summary': 'Five Stars',
 'unixReviewTime': 1477785600}

In [5]:
df = pd.DataFrame(data)

In [6]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"10 30, 2016",A3FO5AKVTFRCRJ,0739079891,francisco,It's good for beginners,Five Stars,1477785600,NaN,NaN,NaN
1,5.0,True,"06 30, 2016",A3UCGC1DHFMBCE,0739079891,Eb Jack Murray,I recommend this starter Ukulele kit. I has e...,Five Stars,1467244800,NaN,NaN,NaN
2,5.0,True,"05 9, 2016",A2S9SLRYLPGYZB,0739079891,Clara LaMarr,G'daughter received this for Christmas present...,Learning new songs to play regularly,1462752000,NaN,NaN,NaN
3,4.0,True,"04 10, 2016",A15RTJWPG8OKOE,0739079891,Eagle80,"According to my order history, I bought this t...",A bargain-bin good-enough ukulele that's held ...,1460246400,NaN,NaN,NaN
4,1.0,True,"02 6, 2016",A12ET1WO3OAVU7,0739079891,Amazon Customer,Please pay attention better than I did to the ...,Poor Quality product.,1454716800,NaN,NaN,NaN


In [7]:
df = df.rename(columns={'asin': 'instrumentID'})

### 1. Предложите 3 способа найти упоминания товаров в отзывах. Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара. Могут помочь заголовки и дополнительные данные с Amazon (Metadata здесь) Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?

1. Скачать метаданные и выделить оттуда сущестивиетльные из title каждого товара. Достоинствами метода являются простота и скорость. Однако этот метод, скорее всего, будет лучше работать на длинных отзывах, потому что с ними выше вероятность, что автор назовёт товар именно нужным нам словом из названия. В целом получается достаточно низкий recall. Можно улучшить этот метод, добавив к названиям общие слова типа product, item и (в нашем случае) instrument.

2. Посчитать самые частотные существительные и свести их с гиперонимами из WordNet. Главное достоинство — всё находится автоматически. Но не для всех слов можно найти синонимы/гиперонимы, также не решается проблема с сокращениями и другими названиями товара.

3. Использование готовых нейросетей. Так мы сможем найти сложные зависимости, но есть риск потерять важные коллокации, потому что будут выделяться только названия брендов, а общие названия продуктов — нет. 

### 2. Реализуйте один из предложенных вами способов.

Я решила реализовать второй способ. 

In [8]:
empty = []
for i, d in enumerate(data):
    try:
        _ = d['reviewText']
    except KeyError:
        empty.append(i)
        
# print(empty)

In [9]:
corpus = [d['reviewText'] for i, d in enumerate(data) if i not in empty]

In [10]:
processed_corpus = []

for text in tqdm(corpus):
    processed_corpus.append(nlp(text))

In [18]:
lemmas = []

for doc in tqdm(processed_corpus):
    for token in doc:
        lemmas.append((token.lemma_, token.pos_))
        
len(lemmas)

15629086

Нахожу самые частотные слова в целом и отдельно смотрю на существительные. 

In [19]:
freq = Counter(lemmas)

top_words_tagged = []
top_nouns_tagged = []

for w_t, f in freq.most_common(5000):
    w, t = w_t
    if w not in stops and w not in string.punctuation:
        top_words_tagged.append(w)
        if t == 'NOUN':
            top_nouns_tagged.append(w)

len(top_words_tagged)

4784

In [20]:
top_nouns_tagged[:50]

['guitar',
 'string',
 'sound',
 'price',
 'pedal',
 'quality',
 'time',
 'one',
 'product',
 'amp',
 'cable',
 'tone',
 'thing',
 'year',
 'case',
 'pick',
 'stand',
 'bass',
 'lot',
 'way',
 'instrument',
 'bit',
 'strap',
 'mic',
 'tuner',
 'problem',
 'set',
 'power',
 'review',
 'effect',
 'music',
 'unit',
 'end',
 'speaker',
 'money',
 'volume',
 'neck',
 'issue',
 'bag',
 'box',
 'work',
 'use',
 'size',
 'star',
 'microphone',
 'light',
 'drum',
 'job',
 'noise',
 'pickup']

Достаю нужные слова из поддерева _musical_instrument.n.01_ в WordNet. 

In [21]:
base = wn.synset('musical_instrument.n.01')
all_names = [s.name() for s in base.hyponyms()]
print(all_names)

['barrel_organ.n.01', 'bass.n.07', 'calliope.n.02', 'electronic_instrument.n.01', "jew's_harp.n.01", 'keyboard_instrument.n.01', 'music_box.n.01', 'percussion_instrument.n.01', 'stringed_instrument.n.01', 'wind_instrument.n.01']


In [15]:
musical_instruments = []
syns = [base]

while syns:
    for syn in syns:
        musical_instruments.append(syn)
        syns.remove(syn)
        syns.extend(syn.hyponyms())

instruments_list = list(set(musical_instruments))
len(instruments_list)

164

In [16]:
all_musical_instruments = []
for i in instruments_list:
    all_musical_instruments.extend(i.lemma_names()) 

len(all_musical_instruments)
# print(all_musical_instruments)

299

Оставляем только те частотные существительные, которые есть в WordNet.

In [22]:
instruments = list(set([i for i in top_words_tagged if i in all_musical_instruments]))
print(instruments)

['recorder', 'chime', 'dulcimer', 'wood', 'grand', 'bongo', 'snare', 'guitar', 'cello', 'piano', 'banjo', 'saxophone', 'wind', 'baritone', 'uke', 'sax', 'tambourine', 'trombone', 'reed', 'drum', 'fiddle', 'violin', 'ukulele', 'triangle', 'organ', 'string', 'bell', 'mandolin', 'whistle', 'pipe', 'instrument', 'harp', 'horn', 'trumpet', 'harmonica', 'clarinet', 'cymbal', 'flute', 'synthesizer', 'bass', 'brass', 'upright']


In [23]:
len(instruments)

42

### 3. Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

In [24]:
left_context = [{'IS_PUNCT': False}, {'LEMMA': {'IN': instruments}}]
right_context = [{'LEMMA': {'IN': instruments}}, {'IS_PUNCT': False}]

context_matcher = Matcher(nlp.vocab)
context_matcher.add('left_context', None, left_context)
context_matcher.add('right_context', None, right_context)

In [25]:
context = []

for doc in tqdm(processed_corpus):
    matches = context_matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        context.append(span.text)
        
len(context)

355482

In [26]:
contexts = Counter(context)
contexts.most_common(10)

[('the guitar', 8934),
 ('the strings', 6238),
 ('my guitar', 5708),
 ('these strings', 4944),
 ('a guitar', 3965),
 ('guitar and', 3635),
 ('your guitar', 3476),
 ('this guitar', 2939),
 ('strings are', 2818),
 ('acoustic guitar', 2559)]

### 4. Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика ранжирует выше коллокации, подходящие для отчёта).

Берём только те биграммы, которые мы выделили в отзывах.

In [27]:
ngrams = []
for c in context:
    n = 0
    words = c.lower().split()
    for word in words:
        if word in stops:
            n += 1
    if n == 0:
        ngrams.append(words)

In [28]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = BigramCollocationFinder.from_documents(ngrams)
bigram_finder.apply_freq_filter(10)

#### PMI

In [29]:
bigram_finder_pmi = bigram_finder.score_ngrams(bigram_measures.pmi)
bigram_finder_pmi[:15]

[(('zenergy', 'chimes'), 11.76051638283185),
 (('hammered', 'dulcimer'), 11.760516382831849),
 (('chimes', 'sat'), 11.526051129194826),
 (('mountain', 'dulcimer'), 11.455412910585846),
 (('bongo', 'cajon'), 10.979156669307187),
 (('hammond', 'organ'), 10.16783284504664),
 (('pitch', 'pipes'), 10.114657840145977),
 (('pitch', 'pipe'), 10.001370553816864),
 (('analog', 'synthesizers'), 9.985582938466619),
 (('note', 'chimes'), 9.965336174720349),
 (('cow', 'bell'), 9.707269256919421),
 (('tin', 'whistle'), 9.686222600484538),
 (('french', 'horn'), 9.441408807006862),
 (('tin', 'whistles'), 9.402964378213763),
 (('foot', 'tambourine'), 9.303624205041343)]

#### likelihood ratio

In [30]:
bigram_finder_likelihood = bigram_finder.score_ngrams(bigram_measures.likelihood_ratio)
bigram_finder_likelihood[:15]

[(('strings', 'guitar'), 10444.070288719724),
 (('guitar', 'bass'), 4569.310104011272),
 (('acoustic', 'guitar'), 4365.147320890128),
 (('great', 'strings'), 4207.626859105092),
 (('12', 'string'), 3602.89323876078),
 (('e', 'string'), 3446.696792048923),
 (('string', 'strings'), 3340.165883991948),
 (('guitar', 'string'), 3075.379070791222),
 (('guitar', 'strap'), 3072.4768184643426),
 (("d'addario", 'strings'), 3039.273111044713),
 (('electric', 'guitar'), 2985.426071593826),
 (('drum', 'machine'), 2981.495152853903),
 (('drum', 'kit'), 2390.2428881263527),
 (('g', 'string'), 2314.294717138254),
 (('drum', 'set'), 2295.4041980755655)]

#### jaccard score

In [31]:
bigram_finder_jaccard = bigram_finder.score_ngrams(bigram_measures.jaccard)
bigram_finder_jaccard[:15]

[(('pitch', 'pipe'), 0.27734375),
 (('mountain', 'dulcimer'), 0.23958333333333334),
 (('zenergy', 'chimes'), 0.17708333333333334),
 (('chimes', 'sat'), 0.1717171717171717),
 (('bongo', 'cajon'), 0.13761467889908258),
 (('alto', 'sax'), 0.1358921161825726),
 (('tin', 'whistle'), 0.13183279742765272),
 (('hammered', 'dulcimer'), 0.12359550561797752),
 (('note', 'chimes'), 0.12318840579710146),
 (('wind', 'screen'), 0.12278481012658228),
 (('analog', 'synthesizers'), 0.11904761904761904),
 (('wind', 'noise'), 0.11902113459399333),
 (('grand', 'piano'), 0.09595100374276964),
 (('pitch', 'pipes'), 0.09523809523809523),
 (('tenor', 'sax'), 0.09230769230769231)]

Мне кажется, лучше всего справляется PMI: выделяется больше качественных сочетаний и есть информация о конкретных брендах, но jaccard score тоже показал неплохие результаты.

### 5. Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров. Должны получиться примерно такие группы:

In [33]:
def get_top(collocations, finder, word):
    reviews = []
    for value in finder:
        if value[0][0] == word or value[0][1] == word:
            reviews.append(value[0][0] + " " + value[0][1])
    return reviews

In [37]:
INSTRUMENTS = ['guitar', 'sax', 'banjo', 'violin', 'bass']

for word in INSTRUMENTS:
    print('\nWord: ', word)
    print(*get_top(contexts, bigram_finder_pmi, word)[:5], sep='\n')
    print()


Word:  guitar
surf guitar
avid guitar
guitar center
guitar processor
guitar accessory


Word:  sax
bari sax
alto sax
sax mouthpiece
tenor sax
soprano sax


Word:  banjo
bluegrass banjo
5-string banjo
6-string banjo
resonator banjo
back banjo


Word:  violin
4/4 violin
violin bow
student violin
violin hanger
violin bridge


Word:  bass
bass traps
beatle bass
bronco bass
bass boost
muddy bass



### 6. Способ объединить синонимичные упоминания (например, "Samsung Galaxy Watch", "watch", "smartwatch")

Можно взять векторные представления названий из метаданных и кластеризовать их. Или можно попробовать решить проблему предобученных моделей с помощью дополнительного обучения на нужных нам текстах, получить новые эмбеддинги и посчитать векторное расстояние. 